In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
df= pd.read_csv("/home/deepika/Documents/NLP/email.csv")

In [4]:
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [5]:
df.isna().sum()

text    0
spam    0
dtype: int64

In [22]:
print(df[df["spam"]==1].count())
print(df[df["spam"]==0].count())

text    1368
spam    1368
dtype: int64
text    4360
spam    4360
dtype: int64


In [5]:
clean_desc = []
for w in range(len(df.text)):
    desc = df['text'][w].lower()
    
    #remove punctuation
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #remove tags
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #remove digits and special chars
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    clean_desc.append(desc)
#assign the cleaned descriptions to the data frame
df['text'] = clean_desc
        
df.head(3)

,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is ...,1
2,subject unbelievable new homes made easy im wa...,1


In [6]:
cv= CountVectorizer()
#vector= cv.fit_transform(text)
#print(vector.toarray())
#print(cv.vocabulary_)

In [7]:
X= cv.fit_transform(df["text"])
y=df["spam"]
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state=40, shuffle= True)

In [9]:
from sklearn.model_selection import StratifiedKFold

In [10]:
kf= StratifiedKFold(n_splits=3)

In [14]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    return(accuracy_score(y_pred, y_test))

In [26]:
logistic=[]
randomforest=[]
gradient=[]


for train, test in kf.split(X,y):
    X_train, X_test, y_train, y_test = X[train],X[test], y[train], y[test]
    
    logistic.append(get_score(LogisticRegression(max_iter=100000), X_train, X_test, y_train, y_test))
    
    randomforest.append(get_score(RandomForestClassifier(), X_train, X_test,y_train, y_test))
    
    gradient.append(get_score(GradientBoostingClassifier(), X_train, X_test,y_train, y_test))

In [27]:
myDict ={0: "Logistic Regression", 1: "RandomForest Classifier",  2: "GradientBoosting Classifier"}

In [31]:
x= [logistic, randomforest, gradient]
accuracy=0
classifier=''
for i, item in enumerate(x):
    print('{} Test accuracy: {}'.format(myDict[i],item[i]))
    if item[i]> accuracy:
        accuracy = item[i]
        classifier= myDict[i]
print("Classifier with best accuracy is:{}".format(classifier))

    

Logistic Regression Test accuracy: 0.9910994764397906
RandomForest Classifier Test accuracy: 0.965950759559979
GradientBoosting Classifier Test accuracy: 0.9732844421162913
Classifier with best accuracy is:Logistic Regression
